# Часть 1. Работа с <a href=https://arxiv.org/abs/2206.00927>DPM-solver</a> в случае flow matching (rectified flow) <br> (5 баллов + 1.0 extra)

В этом ноутбуке вам нужно реализовать три численных солвера для flow macthing: Euler, DPM-1 и DPM-2, и рассмотреть их для различных расписаний (linear, EDM и SD3). Кроме того, вам нужно будет модифицировать DPM-2, добавив multistep режим (двух шаговый метод <a href=https://en.wikipedia.org/wiki/Linear_multistep_method> Адамса</a>).

Пожалуйста, обратите внимание, что вам следует выполнить теоретическое задание, прежде чем приступать к выполнению этого.

------

In [ ]:
!pip install -U diffusers --upgrade
!git clone https://github.com/quickjkee/ysda_hw1

Сначала загрузим пайплайн модели <a href=https://github.com/NVlabs/Sana/tree/main>SANA</a>, которая обучена для генерации картинок по тексту. Данная модель обучена для flow matching случая, предсказывая вектор скорости (velocity prediction)

$x_{t} = (1 - t) x_{0} + t \epsilon$ , $u_{\theta}(x_{t}, t) = \epsilon_{\theta}(x_{t}, t) - x_{\theta}(x_{t}, t), t \in [0, 1]$

In [ ]:
import torch
from diffusers import SanaPipeline
from ysda_hw1.utils.sana_inference import run

pipe = SanaPipeline.from_pretrained(
    "Efficient-Large-Model/Sana_600M_1024px_diffusers",
    variant="fp16",
    torch_dtype=torch.float16,
)
pipe.to("cuda")
pipe.vae.to(torch.float16)
pipe.text_encoder.to(torch.float16)

### Ваш код начинается здесь

Ниже мы определяем две функции: step_fn and schedule_fn.

### Задание 1 (4 балла)
#### 'dpm1_single': 1 балл; 'dpm2_single': 1.5 балла; 'dpm2_multi': 1.5 балла; 

step_fn выполняет один шаг для разных численных схем: 'euler', 'dpm1_single', 'dpm2_single',  'dpm2_multi'. single и multi обознают одношаговый и многошаговый вариант, соответственно. Вам нужно реализовать их.

DPM-солвер подразумевает параметризацию через шум, как это было на лекции. А euler рассматривает дифференциальное уравнение сразу в u парамеризации:

$dx = u(x,t)dt$

### Задание 2 (1 балл)

schedule_fn определяет расписания для семплирования в терминах t (время). Вам нужно рассмотреть три случая: 'linear', 'edm' and 'sd3'. Они определяются по следующим формулам:

обратите внимание что мы следующем общепринятой нотации, обозначая $t$  как $\sigma$ где $\sigma \in [1, 0]$

 <a href=https://github.com/NVlabs/edm/tree/main> EDM</a>:
$ \sigma_{i < N} = \left(\sigma_{max} ^ {\frac{1}{\rho}} + \frac{i}{N-1} \cdot \left(\sigma_{min} ^ {\frac{1}{\rho}} - \sigma_{max} ^ {\frac{1}{\rho}} \right) \right) ^ {\rho}, \sigma_{N} = 0$,

где $N$ - это количество шагов генерации, $\rho$ - гипепараметр, обычно равный $7$. $\sigma_{max}$ и $\sigma_{min}$ - максимальное и минимальное значение времени. Обратите внимание что EDM работает для VE случая ($\sigma_{max}=80$ and $\sigma_{min}=0.002$) когда для FM $\sigma_{max}=1.0$. Однако, мы рекомендуем работать с $\sigma_{max}=80$ и затем масштабировать $\sigma$ в $[0,1]$.

 <a href=https://arxiv.org/html/2403.03206v1>SD3</a>:

$ \sigma_{i} = \frac{s \cdot i}{1 + (s - 1) \cdot i}, i \in [0, 1]$,

где $s$ - это гиперпараметр, контролирующий смещенность к максимальному значению $\sigma$, обычно равняется $3$.

In [ ]:
def step_fn(latents, sigmas, v_pred, i, type_, v_pred_fn, cache=None):
    '''
    Данная функция делает шаг солвера из t в s (t < s): x_t -> x_s
    
    * latents : текущий латент, xt
    * sigmas: временное расписание (дискретные шаги)
    * v_pred: предикт нейронной сети на текущем шаге x_t. 
    В данном случае скорость
    * i: номер текущего шага
    * type_: тип солвера
    * v_pred_fn: функция для предсказания скорости
    * cache: кэш, содержащий предсказания шума для предыдущих шагов
    
    Рекомендации:

    0. sigmas это t. То есть sigmas меняются от 1 до 0.

    1. Для того чтобы делать предсказания моделью используйте v_pred_fn.
       Обратите ванимание что вам надо домножить время на 1000, например
       timestep = (sigmas[i] * 1000).reshape(-1)
       v_pred = v_pred_fn(latents, timestep=timestep).

    2. Обратите внимание на численную стабильность,
       какие то коэффициенты могут принимать большие значения (или равняться бесконечности)
       для начальных и конечных точек.

    3. cache содержит только один элемент
       (шум (epsilon) из предыдущего шага, eps)
    '''
    
    if type_ == 'euler':
        # TODO: your code
        # Euler (solution), delete later

    elif type_ == 'dpm1_single':
        # TODO: your code
        # DPM-1 (solution), delete later

    elif type_ == 'dpm2_single':
        # TODO: your code
        # DPM-2 (solution), delete later

    elif type_ == 'dpm2_multi':
        # TODO: your code
        # DPM-2 (solution), delete later

    else:
        raise ValueError

    return latents


def schedule_fn(n_points, type_, s=3):
    n_points = n_points + 1 # an additional point for t = 0
    
    if type_ == 'linear':
        # TODO: your code
        
    elif type_ == 'sd3':
        # TODO: your code
        # SD3 (example), delete later
        
    elif type_ == 'edm':
        # TODO: your code

    return sigmas

После реализации функции вам необходимо протестировать ее с помощью этой ячейки. Вы можете поиграть с количеством шагов и другими гиперпараметрами. В качестве референса мы привели наши результаты:

<div>
<img src="https://i.postimg.cc/pLQrG8v9/hz.png"/>
</div>


 Обратите внимание, что вам не обязательно получать такие же результаты в точности. Но мы ожидаем от вас правильных выводов:
* Как соотносятся одношаговый и многошаговый варианты для разного числа шагов? Вам нужно выровнять NFE
* Как соотносятся расписания SD3, EDM и linear?
* Как соотносятся DPM-1 и Euler?
* Как соотносятся DPM-2 single и DPM-1 single для разного числа шагов?

Подкрепите свои выводы парными сравнениями сгенерированных картинок

---

Примеры промптов, которые можно рассмотреть


    "portrait photo of a girl, photograph, highly detailed face, depth of field, moody light, golden hour, style by Dan Winters, Steve McCurry, centered, extremely detailed, Nikon D850, award winning photography"
    
    "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k"
    
    'A girl with pale blue hair and a cami tank top'
    
    "Four cows in a pen on a sunny day"
    
    "Three dogs sleeping together on an unmade bed"
    
    "A sky blue colored hippopotamus"
    
    "The interior of a mad scientists laboratory, Cluttered with science experiments, tools and strange machine, Eerie purple light, Close up, by Miyazaki"
    
    "a barred owl peeking out from dense tree branches"
    
    "a close-up of a blue dragonfly on a daffodil"
    
    "A green train is coming down the tracks"
    "A photograph of the inside of a subway train. There are frogs sitting on the seats. One of them is reading a newspaper. The window shows the river in the background."
    
    "a family of four posing at the Grand Canyon"
    
    "A high resolution photo of a donkey in a clown costume giving a lecture at the front of a lecture hall. The blackboard has mathematical equations on it. There are many students in the lecture hall."
    
    "A castle made of tortilla chips, in a river made of salsa. There are tiny burritos walking around the castle"
    
    "A raccoon wearing formal clothes, wearing a tophat and holding a cane. The raccoon is holding a garbage bag. Oil painting in the style of abstract cubism."
    
    "A castle made of cardboard."

In [ ]:
import numpy as np
from functools import partial

image = run(
            pipe,
            step_fn=partial(step_fn, type_='dpm1_single'),  # Play with it
            sigmas=schedule_fn(n_points=10, type_='sd3', s=3),  # Play with it

            prompt="A cat", # You can change the prompt
            height=1024,
            width=1024,
            guidance_scale=4.5,
            generator=torch.Generator(device="cuda").manual_seed(42),
            )[0]
image[0].resize((512, 512))


 ### Задание 3 (1.0 балл)
 
 Кроме того, сгенерируйте 100 изображений, используя промпты из датасета COCO. Рассмотрите два солвера ('dpm2_multi', 'dpm2_single') используя расписание sd3 и 10 шагов (NFE). Аккуратно выровняйте количество NFE для 'dpm2_single' относительно 'dpm2_multi'.

Затем посчитайте <a href="https://github.com/yuvalkirstain/PickScore"> PickScore</a>. Для посчета этой метрики, мы подготовили функцию calc_probs.

 Сделайте это в 2 этапа (чтобы не упасть по памяти): сначала сгенерируйте изображения и сохраните их в переменную, затем посчитайте метрики:

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Generation
metrics_dict = {}
prompts = pd.read_csv('ysda_hw1/utils/data/coco.csv')
prompts = list(prompts['caption'])[:100]

# TODO
# your code here

 Посчитайте метрики (сделайте это батчам, а не все сразу). Приведите усредненные результаты

```
from ysda_hw1.utils.pickscore import load_model, calc_probs

processor, model = load_model() # PickScore model
calc_probs(processor, model, prompt, images)
# images - list of PIL images (batch)
# prompt - list of strings (batch)
```